In [4]:
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import numpy as np
import string
import random
import string
from sklearn import linear_model

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [18]:
def readGz(path):
    for l in gzip.open(path, 'rt'):
        try:
            yield eval(l)
        except:
            continue

In [6]:
def readCSV(path):
    f = gzip.open(path, 'rt')
    f.readline()
    for l in f:
        u,b,r = l.strip().split(',')
        r = int(r)
        yield u,b,r

In [19]:
dataset = []
for d in readGz("renttherunway_final_data.json.gz"):
    dataset.append(d)

In [20]:
len(dataset)

192462